# 编写驱动模块的主体框架


In [ ]:
#include <linux/module.h>
#include <linux/kernel.h>

// 初始化函数,是整个模块的入口函数
int __init mychar_init(void)  // __init 是一个宏定义,表示这个函数是初始化函数,只在初始化时调用
{
    //请注意，模块的初始化函数应该返回一个整数。如果返回0，那么表示初始化成功。
    //如果返回一个负数，那么表示初始化失败，模块将不会被加载。
    
    return 0;
}

// 退出函数,是整个模块的出口函数
void __exit mychar_exit(void) // __exit 是一个宏定义,表示这个函数是退出函数,只在退出时调用
{
    
}

//表示支持GPL的开源协议
MODULE_LICENSE("GPL");

module_init(mychar_init);  // 指定初始化函数 moudule_init是一个宏定义，表示指定初始化函数
module_exit(mychar_exit);  // 指定退出函数 moudule_exit是一个宏定义，表示指定退出函数
//`module_init`是Linux内核中的一个宏，用于指定模块的初始化函数。它的定义在`<linux/init.h>`头文件中
//`module_init`宏接受一个函数名作为参数，这个函数是模块的初始化函数。当模块被加载到内核时，这个函数会被调用
//#define module_init(x)  __initcall(x);

//`module_exit`是Linux内核中的一个宏，用于指定模块的退出函数。它的定义在`<linux/init.h>`头文件中
//`module_exit`宏接受一个函数名作为参数，这个函数是模块的退出函数。当模块被卸载时，这个函数会被调用
//#define module_exit(x)  __exitcall(x);

# 编写字符设备驱动的主体框架


In [ ]:
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/fs.h>
#include <linux/cdev.h>

/************(可选)手动指定主设备号和次设备号************/
int major = 11;  // 主设备号
int minor = 0;  // 次设备号
int mychar_num = 1;  // 设备号的个数


/****************定义cdev对象***********************/
struct cdev mydev;  //这个对象是linux内核对设备的抽象，在这里表示一个字符设备，设备的名称是mydev


/****************定义字符设备的操作函数***************/
int mychar_open(struct inode *pnode, struct file *pfile)
{
    printk("mychar_open is called\n");
    return 0;
}

int mychar_close(struct inode *pnode, struct file *pfile)
{
    printk("mychar_close is called\n");
    return 0;
}


/******************创建file_operations 结构体****************************/
// file_operations结构体是字符设备驱动程序的核心，它包含了驱动程序的入口函数，这些函数是驱动程序的接口
struct file_operations myops = { 
    .owner = THIS_MODULE,  // 这个是必须的，表示这个file_operations对象是属于哪个模块的，THIS_MODULE是一个宏定义，表示当前模块
    .open = mychar_open,  // open函数
    .release = mychar_close,  // close函数
};


/***********************定义初始化函数****************************************/
int __init mychar_init(void) //字符设备的初始化函数
{
    /*************1. 申请设备号 ************/
    if (major)  // 如果主设备号已经被指定
    {
        dev_t devno = MKDEV(major, minor);  // 通过MKDEV宏定义将主设备号和次设备号合成一个dev_t类型的设备号
        int ret = register_chrdev_region(devno, mychar_num, "mychar");  // 通过register_chrdev_region函数申请设备号
        if (ret < 0)  // 如果设备号申请失败
        {
            printk("register_chrdev_region failed\n");
            return ret;
        }
    }
    else  // 如果主设备号没有被指定
    {
        int ret = alloc_chrdev_region(&devno, minor, mychar_num, "mychar");  // 通过alloc_chrdev_region函数动态申请设备号
        if (ret < 0)  // 如果设备号申请失败
        {
            printk("alloc_chrdev_region failed\n");
            return ret;
        }
        major = MAJOR(devno);  // 通过MAJOR宏定义获取主设备号
    }

    /********2.初始化cdev对象************/
    /* 给struct cdev对象指定操作函数集 */
    cdev_init(&mydev, &myops);
    /* 将 struct cdev对象添加到内核对应的数据结构里 */
    mydev.owner = THIS_MODULE;
    int ret = cdev_add(&mydev, devno, mychar_num);
    if (ret < 0)
    {
        printk("cdev_add failed\n");
        return ret;
    }

    return 0;
}

void __exit mychar_exit(void)
{
    cdev_del(&mydev);  // 删除cdev对象
    
    if (major)  // 如果主设备号已经被指定
    {
        dev_t devno = MKDEV(major, minor);
        unregister_chrdev_region(devno, mychar_num);  // 释放设备号
    }
    else  // 如果主设备号没有被指定
    {
        dev_t devno = MKDEV(major, minor);
        unregister_chrdev_region(devno, mychar_num);  // 释放设备号
    }

    
}

//表示支持GPL的开源协议
MODULE_LICENSE("GPL");

module_init(mychar_init);
module_exit(mychar_exit);

## register_chrdev_region 静态注册设备号


`register_chrdev_region`函数是 Linux 内核中的一个函数，用于注册字符设备的主设备号范围。它的原型如下：


In [ ]:
int register_chrdev_region(dev_t first, unsigned int count, const char *name);

该函数接受三个参数：

1. `first`：表示要注册的字符设备的第一个主设备号。
2. `count`：表示要注册的字符设备的数量。
3. `name`：表示要注册的字符设备的名称。

该函数的作用是：将指定的主设备号范围注册到系统中，以便将来可以使用这些主设备号来创建字符设备。注册成功后，其他模块或驱动程序可以使用这些主设备号来创建自己的字符设备。

请注意，`register_chrdev_region`函数在 Linux 内核版本 4.12 之后被废弃，推荐使用`alloc_chrdev_region`函数来注册字符设备的主设备号范围。


## alloc_chrdev_region 动态分配设备号


`alloc_chrdev_region`函数是 Linux 内核中的一个函数，用于`动态分配`字符设备的主设备号范围。它的原型如下：


In [ ]:
int alloc_chrdev_region(dev_t *dev, unsigned int firstminor, unsigned int count, const char *name);

该函数接受四个参数：

1. `dev`：一个指向`dev_t`类型变量的指针，该函数会将分配的第一个设备号存储在这个变量中。
2. `firstminor`：要分配的第一个次设备号。
3. `count`：要分配的设备数量。
4. `name`：要分配的设备的名称。

该函数的作用是动态分配一个主设备号范围，并将分配的第一个设备号存储在`dev`指向的变量中。分配成功后，其他模块或驱动程序可以使用这些设备号来创建自己的字符设备。


In [ ]:

dev_t dev;
if (alloc_chrdev_region(&dev, 0, 1, "mychar"))
    return -ENODEV;

在上面的示例中，我们使用`alloc_chrdev_region`函数分配了一个主设备号和一个次设备号。如果分配失败，我们就返回一个错误码。

## dev_t 与设备号


`dev_t`是 Linux 内核中用于表示设备号的数据类型。设备号是用来唯一标识系统中的设备的。

在 Linux 系统中，每个设备都由一个主设备号和一个次设备号组成。主设备号用于标识设备的类型，比如字符设备或块设备。次设备号用于标识同一类型设备中的具体设备。

`dev_t`类型就是用来存储这两个设备号的。它是一个 32 位的数据类型，其中高 16 位用于存储主设备号，低 16 位用于存储次设备号。

1. 主设备号：占高 12 位，用来表示驱动程序相同的一类设备--摄像头

2. 次设备号：占低 20 位，用来表示被操作的哪个具体设备--前置摄像头/后置摄像头

设备号（32bit）== 主设备号（12bit） + 次设备号（20bit）


`MAJOR`, `MINOR`, 和 `MKDEV` 是在 Linux 内核中定义的宏，用于操作 `dev_t` 类型的设备号。

在 Linux 内核的头文件 `<linux/kdev_t.h>` 中，这些宏的定义如下：


In [ ]:
#define MAJOR(dev) ((unsigned int) ((dev) >> 20))
#define MINOR(dev) ((unsigned int) ((dev) & 0xFFF) | (((dev) >> 12) & ~0xFFF))
#define MKDEV(ma,mi) (((ma) << 20) | ((mi) & 0xFFF) | (((mi) & ~0xFFF) << 12))

这些宏的作用如下：

- `MAJOR(dev)`：从 `dev_t` 类型的设备号 `dev` 中提取出主设备号。
- `MINOR(dev)`：从 `dev_t` 类型的设备号 `dev` 中提取出次设备号。
- `MKDEV(ma, mi)`：使用主设备号 `ma` 和次设备号 `mi` 创建一个 `dev_t` 类型的设备号。

这些宏的使用方法如下：


In [ ]:
dev_t dev = MKDEV(255, 0); // 创建一个主设备号为255，次设备号为0的设备号
int major = MAJOR(dev); // 获取主设备号，结果为255
int minor = MINOR(dev); // 获取次设备号，结果为0

请注意，这些宏的具体定义可能会根据不同的 Linux 内核版本有所不同。上述定义是在较新的 Linux 内核版本中的定义。


## cdev 结构体


`cdev`是 Linux 内核中的一个结构体，用于表示字符设备。它的定义在`<linux/cdev.h>`头文件中，如下所示：


In [ ]:
struct cdev 
{
    struct kobject kobj;  // 内核对象 kobject是内核对象的基类
    struct module *owner;  // 指向拥有这个cdev的模块的指针 module是内核模块的基类
    const struct file_operations *ops; // 指向文件操作函数集的指针 file_operations是文件操作函数集的基类
    struct list_head list;  // 用于将cdev对象添加到内核的数据结构中 list_head是链表的基类
    dev_t dev;  // 设备号
    unsigned int count;  // 设备号的个数
};

这个结构体的成员变量解释如下：

- `kobj`：一个`kobject`结构体，用于在 sysfs 文件系统中表示这个字符设备。
- `owner`：一个指向`module`结构体的指针，表示这个字符设备的所有者模块。
- `ops`：一个指向`file_operations`结构体的指针，表示这个字符设备的文件操作函数集合。
- `list`：一个`list_head`结构体，用于将这个字符设备链接到内核的字符设备链表中。
- `dev`：一个`dev_t`类型的变量，表示这个字符设备的设备号。
- `count`：一个无符号整数，表示这个字符设备的设备数量。


## cdev_init

`cdev_init`函数是 Linux 内核中的一个函数，用于初始化`cdev`结构体。它的原型如下：


In [ ]:
void cdev_init(struct cdev *cdev, const struct file_operations *fops);

该函数接受两个参数：

1. `cdev`：一个指向`cdev`结构体的指针。
2. `fops`：一个指向`file_operations`结构体的指针，表示字符设备的文件操作函数集合。

`cdev_init`函数的作用是初始化`cdev`结构体，并将文件操作函数集合关联到这个`cdev`结构体。

以下是一个示例，演示如何使用`cdev_init`函数初始化`cdev`结构体：


In [ ]:
#include <linux/cdev.h>
#include <linux/fs.h>

struct cdev my_cdev;
struct file_operations my_fops = {
    // 初始化你的文件操作函数集合
};

cdev_init(&my_cdev, &my_fops);


请注意，`file_operations`结构体包含了一组文件操作函数，这些函数定义了字符设备的行为。例如，`open`函数定义了如何打开字符设备，`read`函数定义了如何从字符设备读取数据，`write`函数定义了如何向字符设备写入数据，等等。你需要根据你的字符设备的需求来初始化这个结构体。


## cdev_add

`cdev_add`函数是 Linux 内核中的一个函数，用于将一个已经初始化的`cdev`结构体添加到系统中。它的原型如下：


In [ ]:
int cdev_add(struct cdev *dev, dev_t num, unsigned int count);

该函数接受三个参数：

1. `dev`：一个指向`cdev`结构体的指针。
2. `num`：一个`dev_t`类型的变量，表示字符设备的设备号。
3. `count`：一个无符号整数，表示设备数量。

`cdev_add`函数的作用是将一个已经初始化的`cdev`结构体添加到系统中，使得这个字符设备可以被系统识别和使用。

以下是一个示例，演示如何使用`cdev_add`函数将一个`cdev`结构体添加到系统中：


In [ ]:
#include <linux/cdev.h>
#include <linux/fs.h>

struct cdev my_cdev;
dev_t my_dev = MKDEV(255, 0); // 创建一个设备号
unsigned int count = 1;

int ret = cdev_add(&my_cdev, my_dev, count);
if (ret < 0) {
    printk(KERN_ERR "Failed to add cdev\n");
    return ret;
}


请注意，如果`cdev_add`函数返回一个负数，那么表示添加失败。你应该检查返回值，并处理错误。


## cdev_del 


`cdev_del`函数是 Linux 内核中的一个函数，用于从系统中删除一个`cdev`结构体,使得这个字符设备不再被系统识别和使用。它的原型如下：


In [ ]:
void cdev_del(struct cdev *dev);

该函数接受一个参数：

1. `dev`：一个指向`cdev`结构体的指针。

`cdev_del`函数的作用是从系统中删除一个`cdev`结构体，

以下是一个示例，演示如何使用`cdev_del`函数从系统中删除一个`cdev`结构体：

In [ ]:
#include <linux/cdev.h>

struct cdev my_cdev;

cdev_del(&my_cdev);

请注意，当你不再需要一个字符设备时，你应该使用`cdev_del`函数从系统中删除它。如果你在删除一个字符设备之后还继续使用它，那么可能会导致未定义的行为。


## unregister_chrdev_region

在Linux内核中，如果你使用`register_chrdev_region`函数静态申请了一个设备号或者用`alloc_chrdev_region`函数动态申请了一个设备号，那么当你不再需要这个设备号时，你应该使用`unregister_chrdev_region`函数来释放它。`unregister_chrdev_region`函数的原型如下：

In [ ]:
void unregister_chrdev_region(dev_t dev, unsigned int count);



该函数接受两个参数：

1. `dev`：设备号，这应该是你之前使用`alloc_chrdev_region`函数申请的设备号。
2. `count`：设备数量，这应该与你之前使用`alloc_chrdev_region`函数申请设备号时指定的数量相同。

以下是一个示例，演示如何使用`unregister_chrdev_region`函数释放一个设备号：



In [ ]:
#include <linux/fs.h>

dev_t dev;
unsigned int count = 1;

// 假设你之前使用alloc_chrdev_region函数申请了一个设备号
// dev = MKDEV(major, minor);

unregister_chrdev_region(dev, count);



在上面的示例中，我们首先定义了一个设备号`dev`和一个设备数量`count`。然后，我们调用`unregister_chrdev_region`函数释放设备号。

请注意，你应该在你的驱动程序卸载时释放设备号。如果你在释放设备号之后还继续使用它，那么可能会导致未定义的行为。

`register_chrdev`和`unregister_chrdev`函数在现代的Linux内核中已经被废弃，建议使用`cdev_init`、`cdev_add`和`cdev_del`函数来代替。

# struct file_operation 结构体

在Linux内核中，`file_operations`结构体是用于描述一个字符设备驱动程序的文件操作函数集合的。这个结构体包含了一系列的函数指针，每个函数指针对应一种文件操作，例如`open`、`read`、`write`、`ioctl`等。

当用户空间程序通过设备文件（例如`/dev/mychar`）进行操作时，内核会根据操作的类型调用`file_operations`结构体中对应的函数。例如，当用户空间程序调用`read`系统调用时，内核就会调用`file_operations`结构体的`read`成员。

以下是`file_operations`结构体的定义（部分）：



In [ ]:
struct file_operations {
    struct module *owner;
    loff_t (*llseek) (struct file *, loff_t, int);
    ssize_t (*read) (struct file *, char __user *, size_t, loff_t *);
    ssize_t (*write) (struct file *, const char __user *, size_t, loff_t *);
    int (*open) (struct inode *, struct file *);
    int (*release) (struct inode *, struct file *);
    // 其他成员...
};



在你的驱动程序中，你应该定义一个`file_operations`结构体，并将你的文件操作函数赋值给对应的成员。然后，你可以使用`cdev_init`函数将这个`file_operations`结构体关联到你的`cdev`结构体。例如：



In [ ]:
static int my_open(struct inode *inode, struct file *file)
{
    // 打开设备
    return 0;
}

static struct file_operations my_fops = {
    .open = my_open,
    // 其他成员...
};

struct cdev my_cdev;
cdev_init(&my_cdev, &my_fops);



在上面的示例中，我们首先定义了一个`file_operations`结构体`my_fops`，并将`my_open`函数赋值给`open`成员。然后，我们使用`cdev_init`函数将`my_fops`关联到`my_cdev`。这样，当用户空间程序打开设备文件时，内核就会调用`my_open`函数。

# 操作函数中的两个结构体

struct inode 结构体是随着设备节点创建而在内核中产生的，和设备节点一一对应

struct file 结构体是随着设备文件打开而创建的，会在内核设备描述符中注册一个序号存储在结构体的一个成员里

![alt text](驱动使用端.jpg)

## struct inode

在Linux内核中，`inode`结构体是用于表示文件系统中的一个索引节点（inode）

每个文件（包括目录）在文件系统中都有一个与之关联的索引节点

其中包含了关于该文件的元数据，例如文件大小、文件类型、文件权限、文件的创建时间、最后访问时间、最后修改时间等



`inode`结构体的定义在`<linux/fs.h>`头文件中，它包含了很多成员，以下是一些主要的成员：

- `i_mode`：文件类型和权限。
- `i_uid`：文件的用户ID。
- `i_gid`：文件的组ID。
- `i_size`：文件大小（字节）。
- `i_atime`、`i_mtime`、`i_ctime`：文件的访问时间、修改时间和状态改变时间。
- `i_blocks`：文件所占用的磁盘块数。
- `i_nlink`：文件的硬链接数。

```C
struct inode
{
	//....
	dev_t  i_rdev;        //设备号
	struct cdev  *i_cdev; //如果是字符设备才有此成员，指向对应设备驱动程序中的加入系统的struct cdev对象
	//....
}
```

1. 内核中每个该结构体对象对应着一个实际文件,一对一
2. open一个文件时如果内核中该文件对应的inode对象已存在则不再创建，不存在才创建
3. 内核中用此类型对象关联到对此文件的操作函数集（对设备而言就是关联到具体驱动代码）

在字符设备驱动程序中，`inode`结构体的一个重要用途是在`open`、`release`等文件操作函数中获取设备号。例如：



In [ ]:
static int my_open(struct inode *inode, struct file *file)
{
    int major = imajor(inode);
    int minor = iminor(inode);
    printk(KERN_INFO "Device (%d, %d) opened\n", major, minor);
    return 0;
}



在上面的示例中，我们使用`imajor`和`iminor`函数从`inode`结构体中获取设备的主设备号和次设备号。

除了这个用法，还有一个常用的用法：

因为inode结构体是在内核中伴随设备节点而创建的，所以可以用它在全局中传递`设备描述结构体`的位置

例如：

In [ ]:
struct mychar_dev  //定义设备结构体,属性用每个成员变量表示，这样做目的是为了减少对全局变量的依赖
{
	struct cdev mydev;  //cdev对象
    //下面是设备可能会用到的一些全局变量，这里放到结构体中
	char mydev_buf[BUF_LEN];  //设备的缓冲区
	int curlen;  //缓冲区中有效数据的长度
};

struct mychar_dev gmydev;  //实例化我们需要的完成读写的字符设备对象


在上面的代码中，我们将可能会用到的关于设备的全局变量放到一个专门用来表示设备的一个结构体中，这相当于继承`cdev`类，并创建了一个新的类——`struct mychar_dev`

我们可以看到`mychar_dev`类 有一个 属性 是`cdev`结构体，这也是一个实例化的类，实例化的对象是`mydev`

通过`cdev_add`函数添加到内核中后，`inode`结构体就能获取`mydev`结构体的地址

接下来通过container函数就能获得`gmydev`这个对象的地址，通过这个地址我们就可以很方便地访问结构体的各个属性

```C
container_of(pnode->i_cdev, struct mychar_dev, mydev)
```

实例使用中，我们还可以使用下面的`file`结构体中的private成员存储，以方便在其他函数中调用，例如：


In [ ]:
int mychar_open(struct inode *pnode, struct file *pfile)
{
	pfile->private_data = (void *) (container_of(pnode->i_cdev, struct mychar_dev, mydev)); //因为private_data是一个void指针，这里要将其转换
	printk("mychar_open is called\n");
	return 0;
}
ssize_t mychar_read(struct file *pfile,char __user *puser,size_t count,loff_t *p_pos)
{

	struct mychar_dev *pmydev = (struct mychar_dev *)pfile->private_data;  //本来是void*类型，这里要转换成结构体指针
	int size = 0;
	int ret = 0;

	if(count > pmydev->curlen)  //通过pmydev访问了本来应该定义成全局变量的curlen属性
	{
        //....
    }
    else
    {
        //....
    }
    // ...
}

## struct file 结构体

读写文件内容过程中用到的一些控制性数据组合而成的对象------文件操作引擎（文件操控器）

In [ ]:

struct file
{
	//...
	mode_t f_mode;					//不同用户的操作权限，驱动一般不用
	loff_t f_pos;					//position 数据位置指示器，需要控制数据开始读写位置的设备有用
	unsigned int f_flags;			//open时的第二个参数flags存放在此
	struct file_operations *f_op;	//open时从struct inode中i_cdev的对应成员获得地址，驱动开发中用来协助理解工作原理，内核中使用
	void *private_data;				//本次打开文件的私有数据，驱动中常来在几个操作函数间传递共用数据
	struct dentry *f_dentry;		//驱动中一般不用，除非需要访问对应文件的inode，用法flip->f_dentry->d_inode
    int refcnt;						//引用计数，保存着该对象地址的位置个数，close时发现refcnt为0才会销毁该struct file对象
	//...
};



1. open函数被调用成功一次，则创建一个该对象，因此可以认为一个该类型的对象对应一次指定文件的操作
2. open同一个文件多次，每次open都会创建一个该类型的对象
3. 文件描述符数组中存放的地址指向该类型的对象
4. 每个文件描述符都对应一个struct file对象的地址



# 读写操作的实现

## ssize_t xxx_write 函数

`xxx_write`代表一个字符设备驱动程序中的写操作函数。它的原型如下：



In [ ]:
ssize_t xxx_write(struct file *file, const char __user *buf, size_t count, loff_t *ppos);



这个函数接受四个参数：

1. `*file`：指向open产生的struct file类型的对象，表示本次write对应的那次open
2. `*buf`：指向用户空间一块内存，用来保存被写的数据
3. `count`：用户期望写入的字节数
4. `*ppos`：对于需要位置指示器控制的设备操作有用，用来指示写入的起始位置，写完后也需要变更位置指示器的指示位置

这个函数应该返回值 `ssize_t` 为实际写入的数据的大小。如果发生错误，返回一个负的错误码。

以下是一个简单的示例，演示如何实现`xxx_write`函数：



In [ ]:
ssize_t xxx_write(struct file *file, const char __user *buf, size_t count, loff_t *ppos)
{
    char *kbuf = kmalloc(count, GFP_KERNEL);  // 申请内核缓冲区
    if (!kbuf)
        return -ENOMEM;

    if (copy_from_user(kbuf, buf, count))  // 从用户空间拷贝数据到内核空间
    {                                      
        kfree(kbuf);  // 如果拷贝失败，释放内核缓冲区
        return -EFAULT;
    }

    // 在这里处理kbuf

    kfree(kbuf);
    *ppos += count;
    return count;
}



在上面的示例中，我们首先使用`kmalloc`函数分配了一块内核空间。然后，我们使用`copy_from_user`函数将用户空间的数据复制到内核空间。然后，我们处理这些数据。最后，我们释放内核空间，并更新文件位置。

## ssize_t xxx_read 

`xxx_read`可能是一个字符设备驱动程序中的读操作函数。它的原型应该如下：

In [ ]:
ssize_t mychar_read(struct file *file, char __user *buf, size_t count, loff_t *ppos);



这个函数接受四个参数：

1. `*file`：指向open产生的struct file类型的对象，表示本次read对应的那次open
2. `*buf`：指向用户空间一块内存，用来保存读到的数据
3. `*count`：用户期望读取的字节数
4. `*ppos`：对于需要位置指示器控制的设备操作有用，用来指示读取的起始位置，读完后也需要变更位置指示器的指示位置

这个函数应该返回值`ssize_t`为读取的数据的大小。如果发生错误，你应该返回一个负的错误码。

以下是一个简单的示例，演示如何实现`mychar_read`函数：



In [ ]:
ssize_t mychar_read(struct file *file, char __user *buf, size_t count, loff_t *ppos)
{
    char *kbuf = "Hello, world!\n";
    size_t len = strlen(kbuf);

    if (*ppos >= len)
        return 0;

    if (*ppos + count > len)
        count = len - *ppos;

    if (copy_to_user(buf, kbuf + *ppos, count))
        return -EFAULT;

    *ppos += count;
    return count;
}



在上面的示例中，我们首先定义了一个内核空间的字符串`kbuf`。然后，我们检查文件位置，如果文件位置已经超过了字符串的长度，我们就返回0，表示没有更多的数据可以读取。然后，我们检查缓冲区的大小，如果缓冲区的大小超过了剩余的数据的大小，我们就减小缓冲区的大小。然后，我们使用`copy_to_user`函数将内核空间的数据复制到用户空间。最后，我们更新文件位置，并返回读取的数据的大小。

## put_user 

`put_user(x, ptr)`是一个Linux内核中的宏，用于将一个简单的值从内核空间复制到用户空间。这个宏接受两个参数：

1. `x`：要复制的值。这个值应该是一个简单的类型，例如`int`、`long`、`char`等。
2. `ptr`：一个指向用户空间的指针，表示复制的目标。

这个宏的使用非常简单，你只需要提供要复制的值和目标指针即可。例如：



In [ ]:
int x = 123;
int __user *ptr = ...;
put_user(x, ptr);



在上面的示例中，我们首先定义了一个`int`类型的变量`x`，并将其初始化为123。然后，我们使用`put_user`宏将`x`的值复制到用户空间的`ptr`指向的位置。

注意，`put_user`宏可能会失败，例如当用户空间的指针无效时。因此，你应该总是检查`put_user`宏的返回值，如果返回值不是0，那么就表示发生了错误。例如：



In [ ]:
int x = 123;
int __user *ptr = ...;
if (put_user(x, ptr))
    return -EFAULT;



在上面的示例中，我们使用`if`语句检查了`put_user`宏的返回值，如果返回值不是0，我们就返回一个错误码表示发生了错误。

## unsigned long copy_to_user

`copy_to_user`是Linux内核中的一个函数，用于将数据从内核空间复制到用户空间。这个函数的原型如下：



In [ ]:
unsigned long copy_to_user(void __user *to, const void *from, unsigned long n);



这个函数接受三个参数：

1. `to`：一个指向用户空间的指针，表示复制的目标。
2. `from`：一个指向内核空间的指针，表示复制的源。
3. `n`：要复制的数据的大小（字节）。

这个函数返回未复制的字节数。如果所有的数据都被成功复制，那么返回值就是0。如果有部分数据未被复制，那么返回值就是未复制的数据的大小。

以下是一个简单的示例，演示如何使用`copy_to_user`函数：



In [ ]:
char kbuf[100];
char __user *ubuf = ...;
size_t size = ...;

if (copy_to_user(ubuf, kbuf, size))
    return -EFAULT;



在上面的示例中，我们首先定义了一个内核空间的缓冲区`kbuf`。然后，我们使用`copy_to_user`函数将`kbuf`的内容复制到用户空间的`ubuf`指向的位置。如果`copy_to_user`函数返回非0值，我们就返回一个错误码表示发生了错误。

## __user 标记

在Linux内核编程中，`__user`是一个标记，用于标记指向用户空间的指针。这个标记没有实际的功能，但是可以帮助开发者和静态分析工具识别出哪些指针是指向用户空间的。

在内核代码中，你不能直接访问用户空间的内存，因为这可能会导致内核崩溃。你应该使用特殊的函数，例如`copy_to_user`、`copy_from_user`、`put_user`和`get_user`，来访问用户空间的内存。

以下是一个使用`__user`标记的示例：



In [ ]:
int my_function(int __user *ptr)
{
    int x;
    if (get_user(x, ptr))
        return -EFAULT;
    // 在这里处理x
    return 0;
}



在上面的示例中，`my_function`函数接受一个指向用户空间的`int`指针。我们使用`get_user`函数从用户空间获取一个`int`值，并将其存储在`x`中。如果`get_user`函数失败，我们就返回一个错误码。

# 二、printk



```c
//日志级别
#define	KERN_EMERG	"<0>"	/* system is unusable			*/
#define	KERN_ALERT	"<1>"	/* action must be taken immediately	*/
#define	KERN_CRIT	"<2>"	/* critical conditions			*/
#define	KERN_ERR	"<3>"	/* error conditions			*/

#define	KERN_WARNING	"<4>"	/* warning conditions			*/

#define	KERN_NOTICE	"<5>"	/* normal but significant condition	*/
#define	KERN_INFO	"<6>"	/* informational			*/
#define	KERN_DEBUG	"<7>"	/* debug-level messages			*/

用法：printk(KERN_INFO"....",....)
    
    printk(KERN_INFO"Hello World"); =====> printk("<6>""Hello World") ====> printk("<6>Hello World")
  
```

dmesg --level=emerg,alert,crit,err,warn,notice,info,debug

```c
#define HELLO_DEBUG
#undef PDEBUG
#ifdef HELLO_DEBUG
#define PDEBUG(fmt, args...) printk(KERN_DEBUG fmt, ##args)
#else
#define PDEBUG(fmt, args...)
#endif

```

# 三、多个次设备的支持

每一个具体设备（次设备不一样的设备），必须有一个struct cdev来代表它

cdev_init

cdev.owner赋值

cdev_add

以上三个操作对每个具体设备都要进行

# 多路复用

多路复用为多个进程共用同一个资源提供实现机制

> 在应用层，有如下几个函数可以实现：

| 函数 | 实现方式 | 可以监控描述符的数量 | 实现效率 |
| --- | ------- | ---------------- | ------- |
| select | 位运算实现 | 监控的描述符数量有限（32位机1024,64位机2048）| 效率差 |
| poll | 链表实现 | 监控的描述符数量不限 | 效率差 |
| epoll | 链表实现 | 监控的描述符数量不限 | 效率最高 |

select 实现的大体机制： 

让内核监听多个描述符，阻塞等待有一个或者多个文件描述符，准备就绪。

内核将没有准备就绪的文件描述符，从集合中清掉。

> 在驱动层的对应实现：

该函数与select、poll、epoll_wait函数相对应，协助这些多路监控函数判断本设备是否有数据可读写

```C
unsigned int xxx_poll(struct file *filp, poll_table *wait) 
//函数名初始化给struct file_operations的成员.poll
{
    unsigned int mask = 0;
    /*
    	1. 将所有等待队列头加入poll_table表中
    	2. 判断是否可读，如可读则mask |= POLLIN | POLLRDNORM;
    	3. 判断是否可写，如可写则mask |= POLLOUT | POLLWRNORM;
    */
    
    return mask;
}
```

```c
void poll_wait(struct file *filp, wait_queue_head_t *wait_address, poll_table *p)；
```

- 功能：将等待队列头添加至poll_table表中
- 参数：
    struct file : 设备文件
    Wait_queue_head_t : 等待队列头
    Poll_table : poll_table表
